# py-装饰器

### 函数计时

In [6]:
# 函数计时装饰器装饰器的创建
import time
from functools import wraps

def timer_counter(func):
    '''
    用装饰器实现函数计时
    :param function: 需要计时的函数
    :return: None
    '''
    @wraps(func)
    def function_timer(*args, **kwargs):
        print('[Function: {name} start...]'.format(name = func.__name__))
        t0 = time.time()
        result = func(*args, **kwargs)
        t1 = time.time()
        print('[Function: {name} finished, spent time: {time:.2f}s]'.format(name = func.__name__, time = t1 - t0))
        return result
    return function_timer

@timer_counter
def test_func():
    print("count start ...")
    for i in range(3):
        time.sleep(1)
    print('count end.')


test_func()

[Function: test_func start...]
count start ...
count end.
[Function: test_func finished, spent time: 3.00s]


### 函数自定义超时设置(仅适用linux)

In [3]:
import os
import errno
import signal
from functools import wraps

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            # signal.alarm(seconds)
            # used timer instead of alarm
            signal.setitimer(signal.ITIMER_REAL, seconds) 
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wraps(func)(wrapper)
    return decorator



import time


@timeout(1)
def loop():
    while True:
       pass

try:
    begin = time.time()
    loop()
except TimeoutError as e:
    print("Time elapsed: {:.3f}s".format(time.time() - begin))

AttributeError: module 'signal' has no attribute 'SIGALRM'

In [5]:
import signal

class timeout(object):
    
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
        
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
        
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
        
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

        
import time

with timeout(seconds=3):
    time.sleep(4)

AttributeError: module 'signal' has no attribute 'SIGALRM'

### trace代码跟踪调试信息

In [9]:
import sys,os,linecache

def trace(f):
    
    def globaltrace(frame, why, arg):
        if why == "call": 
            return localtrace
        return None
    
    def localtrace(frame, why, arg):
        if why == "line":
          # record the file name and line number of every trace 
            filename = frame.f_code.co_filename
            lineno = frame.f_lineno
            bname = os.path.basename(filename)
            print("{}({}): {}".format(bname, lineno, linecache.getline(filename, lineno).strip('\r\n')))
        return localtrace
                  
    def _f(*args, **kwds):
        sys.settrace(globaltrace)
        result = f(*args, **kwds)
        sys.settrace(None)
        return result
    return _f
                  
@trace
def xxx():
    print(1)
    print(22)
    print(333)
    
xxx()

<ipython-input-9-04069aa8592d>(28):     print(1)
iostream.py(367):         if self.pub_thread is None:
iostream.py(371):             if not isinstance(string, unicode_type):
iostream.py(374):             is_child = (not self._is_master_process())
iostream.py(301):         return os.getpid() == self._master_pid
iostream.py(376):             self.pub_thread.schedule(lambda : self._buffer.write(string))
iostream.py(200):         if self.thread.is_alive():
threading.py(1112):         assert self._initialized, "Thread.__init__() not called"
threading.py(1113):         if self._is_stopped or not self._started.is_set():
threading.py(508):         return self._flag
threading.py(1115):         self._wait_for_tstate_lock(False)
threading.py(1069):         lock = self._tstate_lock
threading.py(1070):         if lock is None:  # already determined that the C code is done
threading.py(1072):         elif lock.acquire(block, timeout):
threading.py(1116):         return not self._is_stopped
iostream.